# b. 使用大模型 API 对视频进行快速摘要（音频处理）- 精简版

> 指导文章：[15. 用 API 实现 AI 视频摘要：动手制作属于你的 AI 视频助手](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/15.%20用%20API%20实现%20AI%20视频摘要：动手制作属于你的%20AI%20视频助手.md)
>

这是精简的核心代码，非交互版，直接对应于文章内容，变量名相比于交互版会稍微修改，以更清晰的反映作用。

![Pipeline](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/assets/image-20240926154540268.png?raw=1)

你可以一键执行直接看到结果。

如果你想在本地试试参数交互，下载：[a. 交互完整版](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Demos/13.%20轻松开始你的第一次%20AI%20视频总结（API%20版）.ipynb)

这里还有一个简单的 Python[🎡脚本AI Summarizer 脚本](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/CodePlayground/summarizer.py)供你尝试。命令行执行：
```bash
python summarizer.py ./example_video.mp4
```

## 第 1 部分 - 准备工作


### 安装和导入

In [2]:
!pip install srt==3.5.3
!pip install datasets==2.20.0
!pip install DateTime==5.5
!pip install opencv-contrib-python==4.8.0.76
!pip install opencv-python==4.8.0.76
!pip install opencv-python-headless==4.10.0.84
!pip install openpyxl==3.1.4
!pip install openai==1.35.3
!pip install git+https://github.com/openai/whisper.git@ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
!pip install numpy==1.25.2
!pip install soundfile==0.12.1
!pip install librosa==0.10.2.post1
!pip install httpx==0.27.2

  Preparing metadata (setup.py) ... done
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22428 sha256=9a856c2d65f49ad225eef8e4c742071504437df8064e33df593823ec382ac234
  Stored in directory: /root/.cache/pip/wheels/1f/43/f1/23ee9119497fcb57d9f7046fbf34c6d9027c46a1fa7824cf08
Successfully built srt
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.3 MB/s eta 0:00:00
  Attempting uninstall: soundfile
    Found existing installation: soundfile 0.13.0
    Uninstalling soundfile-0.13.0:
      Successfully uninstalled soundfile-0.13.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [1]:
# 标准库
import os
import time
import re
import pathlib
import textwrap
import datetime

# 第三方库
import numpy as np
import srt
import soundfile as sf
from tqdm import tqdm

# 项目相关库
import whisper
from datasets import load_dataset
from openai import OpenAI

### 加载数据


In [3]:
# 加载数据集
dataset_name = "kuanhuggingface/NTU-GenAI-2024-HW9"
dataset = load_dataset(dataset_name)

# 准备音频
input_audio = dataset["test"]["audio"][0]
input_audio_name = input_audio["path"]
input_audio_array = input_audio["array"].astype(np.float32)
sampling_rate = input_audio["sampling_rate"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [4]:
input_audio

{'path': 'ntu-gen-ai-2024-hw9-16k.mp3',
 'array': array([-0.01757073, -0.01902751, -0.01259745, ...,  0.00020729,
         0.00039151,  0.00021348]),
 'sampling_rate': 16000}

## 第 2 部分 - 自动语音识别 (ASR)

我们将使用 OpenAI 的 Whisper 模型将音频转换为字幕。

下图是处理的样例过程，对应于之后所使用的音频：

![image-20240926155512340](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/assets/image-20240926155512340.png?raw=1)

### 定义语音识别函数

In [5]:
def speech_recognition(model_name, input_audio, output_subtitle_path, decode_options, cache_dir="./"):
    # 加载模型
    model = whisper.load_model(name=model_name, download_root=cache_dir)

    # 转录音频
    transcription = model.transcribe(
        audio=input_audio,
        language=decode_options["language"],
        verbose=False,
        initial_prompt=decode_options["initial_prompt"],
        temperature=decode_options["temperature"]
    )

    # 处理转录结果，生成字幕文件
    subtitles = []
    for i, segment in enumerate(transcription["segments"]):
        start_time = datetime.timedelta(seconds=segment["start"])
        end_time = datetime.timedelta(seconds=segment["end"])
        text = segment["text"]
        subtitles.append(srt.Subtitle(index=i, start=start_time, end=end_time, content=text))

    srt_content = srt.compose(subtitles)

    # 保存字幕文件
    with open(output_subtitle_path, "w", encoding="utf-8") as file:
        file.write(srt_content)

    print(f"字幕已保存到 {output_subtitle_path}")

### 设置参数

注意，这里设置的参数都是 Whisper 相关的，与后续的 AI 摘要不同。

你可以通过 model_name 设置不同的模型，使用**标识**指定。通过[官方仓库](https://github.com/openai/whisper)所提供的数据，我们可以看到不同模型需要的显存大小：

| Size   | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
| ------ | ---------- | ------------------ | ------------------ | ------------- | -------------- |
| tiny   | 39 M       | `tiny.en`          | `tiny`             | ~1 GB         | ~32x           |
| base   | 74 M       | `base.en`          | `base`             | ~1 GB         | ~16x           |
| small  | 244 M      | `small.en`         | `small`            | ~2 GB         | ~6x            |
| medium | 769 M      | `medium.en`        | `medium`           | ~5 GB         | ~2x            |
| large  | 1550 M     | N/A                | `large`            | ~10 GB        | 1x             |

**解释：**

- **Size (大小)**：表示模型的尺寸，不同大小的模型训练时使用的数据量不同，因此性能和精度也不同。较大的模型通常会有更高的精度。`Medium` 是个不错的选择，`tiny` 和 `base` 效果一般，用于学习的话也可以。
- **Parameters (参数量)**：模型的参数数量，表示模型的复杂度。参数越多，模型的性能通常越好，但也会占用更多的计算资源。
- **English-only model (仅限英文模型)**：模型的**标识**名称，只用于处理英文音频转录，适用于仅需要处理英文语音的场景。
- **Multilingual model (多语言模型)**：模型的**标识**名称，用于在代码中加载相应的模型，对应于接下来的 `model_name` 参数。
- **Required VRAM (所需显存)**：指运行该模型时所需的显存大小。如果你对参数和显存的对应关系感兴趣，可以阅读之前的文章：[《07. 探究模型参数与显存的关系以及不同精度造成的影响.md》](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/07.%20探究模型参数与显存的关系以及不同精度造成的影响.md)。
- **Relative speed (相对速度)**：相对速度表示模型处理语音转录任务的效率。数字越大，模型处理速度越快，与模型的参数量成反比。

In [6]:
# 模型名称，可选 'tiny', 'base', 'small', 'medium', 'large-v3'
model_name = 'medium'

# 语言
language = 'zh'  # 选择语音识别的目标语言，如 'zh' 表示中文

# 初始 prompt，可选
initial_prompt = '请用中文'  # 如果需要，可以为 Whisper 模型设置初始 prompt 语句

# 采样温度，控制模型的输出多样性
temperature = 0.0  # 0 表示最确定性的输出，范围为 0-1

# 输出文件后缀
suffix = '信号与人生'

# 字幕文件路径
output_subtitle_path = f"./output-{suffix}.srt"

# 模型缓存目录
cache_dir = './'

### 运行语音识别


In [7]:
# 构建解码选项
decode_options = {
    "language": language,
    "initial_prompt": initial_prompt,
    "temperature": temperature
}

# 运行 ASR
speech_recognition(
    model_name=model_name,
    input_audio=input_audio_array,
    output_subtitle_path=output_subtitle_path,
    decode_options=decode_options,
    cache_dir=cache_dir
)

100%|█████████████████████████████████████| 1.42G/1.42G [00:16<00:00, 92.2MiB/s]
100%|██████████| 104500/104500 [02:48<00:00, 619.33frames/s]

字幕已保存到 ./output-信号与人生.srt


In [8]:
segment

NameError: name 'segment' is not defined

### 检查结果


In [9]:
# 读取并打印字幕内容
with open(output_subtitle_path, 'r', encoding='utf-8') as file:
    content = file.read()
print(content)

1
00:00:00,000 --> 00:00:04,000
每次说 学问是做出来的

2
00:00:06,000 --> 00:00:08,000
什么意思

3
00:00:08,000 --> 00:00:12,000
要做 才会获得学问

4
00:00:13,000 --> 00:00:16,000
你如果每天光是坐在那里听

5
00:00:17,000 --> 00:00:20,000
学问很可能是左耳近右耳出的

6
00:00:21,000 --> 00:00:23,000
你光是坐在那儿读

7
00:00:23,000 --> 00:00:26,000
学问可能从眼睛进入脑海之后就忘掉了

8
00:00:26,000 --> 00:00:29,000
如何能够学问在脑海里面

9
00:00:31,000 --> 00:00:33,000
真的变成你自己学问

10
00:00:33,000 --> 00:00:35,000
就是要做

11
00:00:36,000 --> 00:00:39,000
可能有很多同学有这个经验

12
00:00:39,000 --> 00:00:41,000
你如果去修某一门课

13
00:00:41,000 --> 00:00:44,000
或者做某一个实验

14
00:00:44,000 --> 00:00:47,000
在期末就是要教一个final project

15
00:00:48,000 --> 00:00:50,000
那个final project就是要你把

16
00:00:51,000 --> 00:00:53,000
学到的很多东西

17
00:00:53,000 --> 00:00:56,000
最后整合在你的final project里面

18
00:00:56,000 --> 00:00:58,000
最后做出来的时候

19
00:00:58,000 --> 00:01:00,000
就是把它们都整合了

20
00:01:00,000 --> 00:01:02,000
当你学期结束

21
00:01:02,000 --> 00:01:04,000
真的把final project做完的时候

22
00:01:04,000 --> 00:01:05,000

## 第 3 部分 - 处理自动语音识别的结果

### 提取字幕文本

In [10]:
def extract_and_save_text(srt_filename, output_filename):
    # 读取 SRT 文件
    with open(srt_filename, 'r', encoding='utf-8') as file:
        content = file.read()

    # 去除时间戳和索引
    pure_text = re.sub(r'\d+\n\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}\n', '', content)
    pure_text = re.sub(r'\n\n+', '\n', pure_text)

    # 保存纯文本
    with open(output_filename, 'w', encoding='utf-8') as output_file:
        output_file.write(pure_text)

    print(f'提取的文本已保存到 {output_filename}')

    return pure_text

### 拆分文本



In [11]:
def chunk_text(text, max_length):
    return textwrap.wrap(text, max_length)

### 执行文本处理



In [12]:
# 文本块长度
chunk_length = 512

# 是否转换为繁体中文
convert_to_traditional = False

# 提取文本并拆分
pure_text = extract_and_save_text(
    srt_filename=output_subtitle_path,
    output_filename=f"./output-{suffix}.txt",
)


chunks = chunk_text(text=pure_text, max_length=chunk_length)

提取的文本已保存到 ./output-信号与人生.txt


In [13]:
chunks  # list[str,str,str...]  每一个str是512长度的

['每次说 学问是做出来的 什么意思 要做 才会获得学问 你如果每天光是坐在那里听 学问很可能是左耳近右耳出的 你光是坐在那儿读 学问可能从眼睛进入脑海之后就忘掉了 如何能够学问在脑海里面 真的变成你自己学问 就是要做 可能有很多同学有这个经验 你如果去修某一门课 或者做某一个实验 在期末就是要教一个final project 那个final project就是要你把 学到的很多东西 最后整合在你的final project里面 最后做出来的时候 就是把它们都整合了 当你学期结束 真的把final project做完的时候 你会忽然发现 我真的学到很多东西 那就是做出来的学问 也许可以举另外一个例子 就是你如果学了某一些 很复杂的演算法或者什么 好像觉得那些不见得在你的脑海里 可是后来老师出了个习题 那个习题叫你写一个很大的程式 要把所有东西都包进去 当你把这个程式写完的时候 你会发现 你忽然把演算法 所有东西都弄通了 那就是学问是做出来的 所以我们永远要记得 尽量多动手多做 在动手跟做的过程之中 学问才可以变成是自己的 同样的情形就是说 很多时候这样动手或者做的 表现或者成绩 没有一个成绩单上的数字',
 '使得很多人觉得那不重要 很多人甚至觉得 这门课要做final project 我就不修了太累了 或者说那门课需要 怎么样太累我就不要做了 而不知道 其实那个才是让你做的机会 然后可以学到最多 也就是说虽然很可能 那么辛苦的做很多事 没有让你获得什么具体成绩 对你的overfitting可能没有帮助 可是对你的全面学习 是很有帮助是该学的 不要漏掉这些事 这是我所说的 这个课业内可以做的这些事 刚才我们讲到思考的时候 我觉得我漏掉一点 你如果修我的信号课 你可能会发现 我上课没讲到一个数学试字的时候 我通常都不推它的 我是在解释那个数学试字在说什么话 同样的没讲到一个什么事情的时候 我通常就在解释他在说什么话 也就是说我在讲的就是 我读到特本那里的时候 我心里怎么想的 也就是我在告诉同学 如何读书的时候 如何一面读一面练习思考 这个才是最重要的一件事 如何培养自己思考的能力 跟培养思考的习惯 我觉得最好的办法就是 读书的时候凡是读到一个数学试字 都去想一想 那个数学试字到底在说什么 凡是读到特本上讲什么 就去想一想 那个到底在说什么 你要真的了解他在说什么的时候

## 第 4 部分 - 文本摘要

### 设置 OpenAI API

首先，需要设置 OpenAI API 密钥，这里使用的是阿里云的大模型，你可以通过[《00. 大模型 API 获取步骤》](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/00.%20大模型%20API%20获取步骤.md)获取 API 密钥。

如果需要使用其他平台，请参考对应的开发文档后对应修改 base_url。

In [15]:
!pip list  # httpx到达0.28.0以上会导致Openai 报错Client.__init__() got an unexpected keyword argument 'proxies'

Package                            Version
---------------------------------- ------------------
absl-py                            1.4.0
accelerate                         1.2.1
aiohappyeyeballs                   2.4.4
aiohttp                            3.11.11
aiosignal                          1.3.2
alabaster                          1.0.0
albucore                           0.0.19
albumentations                     1.4.20
altair                             5.5.0
annotated-types                    0.7.0
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.6.0
arviz                              0.20.0
astropy                            6.1.7
astropy-iers-data                  0.2025.1.6.0.33.42
astunparse                         1.6.3
atpublic                           4.1.0
attrs                              24.3.0
audioread                          3.0.1
autograd             

In [3]:
!pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [17]:
!pip install openai==1.35.3
# !pip install openai

  Using cached openai-1.35.3-py3-none-any.whl.metadata (21 kB)
Using cached openai-1.35.3-py3-none-any.whl (327 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.55.3
    Uninstalling openai-1.55.3:
      Successfully uninstalled openai-1.55.3


In [15]:
!pip install httpx==0.27.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.1
    Uninstalling httpx-0.27.1:
      Successfully uninstalled httpx-0.27.1


In [25]:
!pip list

Package                            Version
---------------------------------- ------------------
absl-py                            1.4.0
accelerate                         1.2.1
aiohappyeyeballs                   2.4.4
aiohttp                            3.11.11
aiosignal                          1.3.2
alabaster                          1.0.0
albucore                           0.0.19
albumentations                     1.4.20
altair                             5.5.0
annotated-types                    0.7.0
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.6.0
arviz                              0.20.0
astropy                            6.1.7
astropy-iers-data                  0.2025.1.6.0.33.42
astunparse                         1.6.3
atpublic                           4.1.0
attrs                              24.3.0
audioread                          3.0.1
autograd             

In [2]:
# 设置 OpenAI API 密钥
openai_api_key = 'sk-8a60a911c43e48afba1e0f7f21630b4f'

# 构建 OpenAI 客户端
client = OpenAI(
    api_key=openai_api_key,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", # 这里使用的是阿里云的大模型，如果需要使用其他平台，请参考对应的开发文档后对应修改。如果使用 GPT 的 API，删除这行就可以直接运行。
)


### 设置参数

默认使用 qwen-turbo，其他模型可以参阅[模型广场 -- 阿里云百炼](https://bailian.console.aliyun.com/?spm=5176.29619931.J__Z58Z6CX7MY__Ll8p1ZOR.1.4d1d59fcWwSqvr#/model-market)，点击对应模型的`查看详情`。

![image-20240924091151684](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/assets/image-20240924091151684.png?raw=1)

在界面可以左上角看到对应的英文名称，复制它，然后替换 `model_name`。

![image-20240924091414350](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/assets/image-20240924091414350.png?raw=1)

你可以随意更换为你想要的模型，不过可能要先申请使用（通过大概要几个小时，会有短信prompt）。

In [14]:
# 模型名称
model_name = 'qwen-turbo'

# 控制响应的随机性
temperature = 0.0

# 控制多样性
top_p = 1.0

# 最大生成tokens
max_tokens = 512

### 定义摘要函数


In [15]:
def summarization(client, summarization_prompt, model_name="qwen-turbo", temperature=0.0, top_p=1.0, max_tokens=512):
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": summarization_prompt}],
        model=model_name,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

### 这里演示两种摘要方式

分别对应于 `MapReduce` 和 `Refine`，你可以通过接下来的代码来感受二者的区别。

#### 方法一：拆分为多段进行摘要（Multi-Stage Summarization）- MapReduce

![image.png](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/assets/image-20240924092040340.png?raw=1)

1. 将长文本分成多个较小的部分，并分别获取每个小段落的摘要

In [16]:
# 定义摘要prompt模板
summarization_prompt_template = "用 300 个字以内写出这段视频文本的摘要，其中包括要点和所有重要细节：<text>"

# 对每个文本块生成摘要
paragraph_summaries = []
for index, chunk in enumerate(chunks):
    print(f"\n========== 正在生成第 {index + 1} 段摘要 ==========\n")
    print(f"原始文本 (第 {index + 1} 段):\n{chunk}\n")

    # 构建摘要prompt
    summarization_prompt = summarization_prompt_template.replace("<text>", chunk)

    # 调用摘要函数
    summary = summarization(
        client=client,
        summarization_prompt=summarization_prompt,
        model_name=model_name,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens
    )

    # 打印生成的摘要
    print(f"生成的摘要 (第 {index + 1} 段):\n{summary}\n")

    # 将生成的摘要保存到列表
    paragraph_summaries.append(summary)


========== 正在生成第 1 段摘要 ==========

原始文本 (第 1 段):
每次说 学问是做出来的 什么意思 要做 才会获得学问 你如果每天光是坐在那里听 学问很可能是左耳近右耳出的 你光是坐在那儿读 学问可能从眼睛进入脑海之后就忘掉了 如何能够学问在脑海里面 真的变成你自己学问 就是要做 可能有很多同学有这个经验 你如果去修某一门课 或者做某一个实验 在期末就是要教一个final project 那个final project就是要你把 学到的很多东西 最后整合在你的final project里面 最后做出来的时候 就是把它们都整合了 当你学期结束 真的把final project做完的时候 你会忽然发现 我真的学到很多东西 那就是做出来的学问 也许可以举另外一个例子 就是你如果学了某一些 很复杂的演算法或者什么 好像觉得那些不见得在你的脑海里 可是后来老师出了个习题 那个习题叫你写一个很大的程式 要把所有东西都包进去 当你把这个程式写完的时候 你会发现 你忽然把演算法 所有东西都弄通了 那就是学问是做出来的 所以我们永远要记得 尽量多动手多做 在动手跟做的过程之中 学问才可以变成是自己的 同样的情形就是说 很多时候这样动手或者做的 表现或者成绩 没有一个成绩单上的数字

生成的摘要 (第 1 段):
这段视频讨论了“学问是做出来的”这句话的含义。它指出，仅仅听讲或阅读并不会真正掌握知识，而通过实践和实际操作才能将知识内化为自己的学问。视频提到了修课和实验中最终项目（final project）的重要性，通过完成这些项目，学生可以把所学知识进行整合，从而更好地理解和掌握。另一个例子是学习复杂算法时，通过解决实际问题或编写程序，学生才能真正理解并掌握这些知识。视频强调，要尽量多动手、多实践，因为只有通过实际操作，学问才能真正变成自己的东西。最后，视频指出这种通过实践获取的知识往往比成绩单上的分数更能体现一个人的能力。


========== 正在生成第 2 段摘要 ==========

原始文本 (第 2 段):
使得很多人觉得那不重要 很多人甚至觉得 这门课要做final project 我就不修了太累了 或者说那门课需要 怎么样太累我就不要做了 而不知道 其实那个才是让你做的机会 然后可以学到最多 也就是说虽然很可能 那么辛苦的做很多

2. 在分别获取每个小段落的摘要后，处理这些摘要以生成最终的摘要。

In [19]:
# 合并段落摘要
collected_summaries = "\n".join(paragraph_summaries)

# 定义最终摘要prompt模板
final_summarization_prompt = "在 500 字以内写出以下文字的简洁摘要，不要分点：<text>"
final_summarization_prompt = final_summarization_prompt.replace("<text>", collected_summaries)

# 生成最终摘要
final_summary = summarization(
    client=client,
    summarization_prompt=final_summarization_prompt,
    model_name=model_name,
    temperature=temperature,
    top_p=top_p,
    max_tokens=max_tokens
)

print(final_summary)

这段视频探讨了多方面的学习和成长，涵盖学业、人际关系和职业技能。视频首先强调了实践和动手操作在学习中的核心作用，认为通过Final Project、实际解决问题和编程等实践手段，学生能更好地掌握知识。同时，视频提倡在课外活动中学习，无论打球、爬山还是参与社团活动，都是促进全人发展的途径。视频还指出，电机工程领域的成功不仅依赖专业知识，还需要良好的软技能，如团队合作和沟通能力。最后，视频分析了影响个人发展的因素，包括实力、努力、自我技能和大致方向，强调全面发展的重要性，并鼓励通过不断学习和实践来提升自我。


#### 方法二：精炼方法（the method of Refinement) - Refine

Refinement 就是把每次的文本和之前的摘要结合起来丢给大模型，类似于迭代：

![Refinement](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/assets/image-20240924092753352.png?raw=1)

步骤（Pipeline）如下：
- 第1步：从一小部分数据开始，运行prompt生成初始输出。
- 第2步：对后续每个文档，将前一个输出与新文档结合输入。
- 第3步：LLM 根据新文档中的信息精炼输出。
- 第4步：此过程持续迭代，直到处理完所有文档。

对应的核心代码：

In [20]:
# 定义初始摘要prompt模板
summarization_prompt_template = "用 300 个字以内写出这段视频文本的摘要，其中包括要点和所有重要细节:<text>"

# 定义精炼摘要prompt模板
summarization_prompt_refinement_template = "请在 500 字以内，结合原先的摘要和新的内容，提供简洁的摘要:<text>"

# 初始化保存摘要的列表
refined_summaries = []

# 对文本块逐步进行精炼摘要，并打印中间过程
for index, chunk in enumerate(chunks):
    if index == 0:
        # 第一步：对第一段文本生成初始摘要
        print(f"\n========== 正在生成第 {index + 1} 段的初始摘要 ==========\n")
        print(f"原始文本 (第 {index + 1} 段):\n{chunk}\n")

        # 构建初始摘要prompt
        summarization_prompt = summarization_prompt_template.replace("<text>", chunk)

        # 调用摘要函数生成第一个摘要
        first_summary = summarization(
            client=client,
            summarization_prompt=summarization_prompt,
            model_name=model_name,
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens
        )

        # 打印生成的初始摘要
        print(f"生成的摘要 (第 {index + 1} 段):\n{first_summary}\n")

        # 保存生成的摘要
        refined_summaries.append(first_summary)

    else:
        # 后续步骤：结合前一个摘要与当前段落进行精炼
        print(f"\n========== 正在生成第 {index + 1} 段的精炼摘要 ==========\n")
        print(f"原始文本 (第 {index + 1} 段):\n{chunk}\n")

        # 构建精炼摘要的输入文本，将前一个摘要与当前段落内容结合
        chunk_with_previous_summary = f"前 {index} 段的摘要: {refined_summaries[-1]}\n第 {index + 1} 段的内容: {chunk}"

        # 构建精炼摘要prompt
        summarization_prompt = summarization_prompt_refinement_template.replace("<text>", chunk_with_previous_summary)

        # 调用摘要函数生成精炼摘要
        refined_summary = summarization(
            client=client,
            summarization_prompt=summarization_prompt,
            model_name=model_name,
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens
        )

        # 打印生成的精炼摘要
        print(f"生成的摘要 (第 {index + 1} 段):\n{refined_summary}\n")

        # 保存生成的精炼摘要
        refined_summaries.append(refined_summary)

# 最终的精炼摘要结果就是 refined_summaries 列表的最后一个元素
final_refined_summary = refined_summaries[-1]

print("\n========== 最终精炼摘要结果 ==========\n")
print(final_refined_summary)


========== 正在生成第 1 段的初始摘要 ==========

原始文本 (第 1 段):
每次说 学问是做出来的 什么意思 要做 才会获得学问 你如果每天光是坐在那里听 学问很可能是左耳近右耳出的 你光是坐在那儿读 学问可能从眼睛进入脑海之后就忘掉了 如何能够学问在脑海里面 真的变成你自己学问 就是要做 可能有很多同学有这个经验 你如果去修某一门课 或者做某一个实验 在期末就是要教一个final project 那个final project就是要你把 学到的很多东西 最后整合在你的final project里面 最后做出来的时候 就是把它们都整合了 当你学期结束 真的把final project做完的时候 你会忽然发现 我真的学到很多东西 那就是做出来的学问 也许可以举另外一个例子 就是你如果学了某一些 很复杂的演算法或者什么 好像觉得那些不见得在你的脑海里 可是后来老师出了个习题 那个习题叫你写一个很大的程式 要把所有东西都包进去 当你把这个程式写完的时候 你会发现 你忽然把演算法 所有东西都弄通了 那就是学问是做出来的 所以我们永远要记得 尽量多动手多做 在动手跟做的过程之中 学问才可以变成是自己的 同样的情形就是说 很多时候这样动手或者做的 表现或者成绩 没有一个成绩单上的数字

生成的摘要 (第 1 段):
这段视频主要强调“学问是做出来的”。它指出，仅仅听或读书不能有效掌握知识，真正理解并记住所学内容需要通过实践。视频提到期末项目（final project）作为学习成果的综合体现，让学生将所学知识应用其中，从而更深刻地理解和掌握。此外，视频还举例说明了解决复杂编程问题时，通过实际编写代码来理解算法的过程。总的来说，视频提倡通过动手实践来巩固知识，并认为这是将所学知识内化为自身能力的有效方法。学习成绩单上的分数并不能完全反映一个人的实际能力和学习深度。


========== 正在生成第 2 段的精炼摘要 ==========

原始文本 (第 2 段):
使得很多人觉得那不重要 很多人甚至觉得 这门课要做final project 我就不修了太累了 或者说那门课需要 怎么样太累我就不要做了 而不知道 其实那个才是让你做的机会 然后可以学到最多 也就是说虽然很可能 那么辛苦的做很多事 没有让你获得什么具体成绩 对你的overfittin